In [1]:
import fastjet as fj
import fjcontrib as rt
import fjext

import pythia8
import pythiafjext as pyfj
import pythiaext

In [14]:
import sys
!{sys.executable} -m pip install ipympl
!{sys.executable} -m pip install ipywidgets
# note you may need to restart the lab after that
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from tqdm.notebook import tqdm

In [3]:
def create_and_init_pythia(config_strings=[]):
    pythia = pythia8.Pythia()
    for s in config_strings:
        pythia.readString(s)
    for extra_s in ["Next:numberShowEvent = 0", "Next:numberShowInfo = 0", "Next:numberShowProcess = 0"]:
        pythia.readString(extra_s)
    if pythia.init():
        return pythia
    return None

In [4]:
sconfig_pythia = [ "Beams:eCM = 8000.", "HardQCD:all = on", "PhaseSpace:pTHatMin = 20."]
pythia = create_and_init_pythia(sconfig_pythia)

In [5]:
# set up our jet definition and a jet selector
jet_R0 = 0.4
jet_def = fj.JetDefinition(fj.antikt_algorithm, jet_R0)
selector = fj.SelectorPtMin(20.0) & fj.SelectorPtMax(40.0) & fj.SelectorAbsEtaMax(1)
sd = rt.SoftDrop(0, 0.1, 1.0)

In [6]:
# set up our jet definition and a jet selector
jet_R0 = 0.4
jet_def = fj.JetDefinition(fj.antikt_algorithm, jet_R0)
jet_selector = fj.SelectorPtMin(20.0) & fj.SelectorPtMax(40.0) & fj.SelectorAbsEtaMax(1)
sd = rt.SoftDrop(0, 0.1, 1.0)

In [15]:
all_jets = []
# for iEvent in tqdm.notebook.tqdm(range(100), 'event'):
for iEvent in tqdm(range(100), 'event'):
    if not pythia.next(): continue
    parts = pyfj.vectorize(pythia, True, -1, 1, False)
    jets = jet_selector(jet_def(parts))
    all_jets.extend(jets)

event:   0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
def deltas(jets, jets0):
    for i in range(len(jets)):
        yield jets0[i].perp() - jets[i].perp()

In [17]:
%%time
all_sd_jets = [sd.result(j) for j in all_jets]

etas = [j.eta() for j in all_jets]
pts = [j.pt() for j in all_jets]
sd_pts = [j.pt() for j in all_sd_jets]
sd_delta_pt = [delta for delta in deltas(all_jets, all_sd_jets)]

angs0 = [fjext.angularity(j, 0., 1.) for j in all_jets]
sd_angs0 = [fjext.angularity(j, 0., 1.) for j in all_sd_jets]
angs0_R0 = [fjext.angularity(j, 0., jet_R0) for j in all_jets]
sd_angs0_R0 = [fjext.angularity(j, 0., jet_R0) for j in all_sd_jets]

angs1 = [fjext.angularity(j, 1., 1.) for j in all_jets]
sd_angs1 = [fjext.angularity(j, 1., 1.) for j in all_sd_jets]
angs1_R0 = [fjext.angularity(j, 1., jet_R0) for j in all_jets]
sd_angs1_R0 = [fjext.angularity(j, 1., jet_R0) for j in all_sd_jets]

angs15 = [fjext.angularity(j, 1.5, 1.) for j in all_jets]
sd_angs15 = [fjext.angularity(j, 1.5, 1.) for j in all_sd_jets]
angs15_R0 = [fjext.angularity(j, 1.5, jet_R0) for j in all_jets]
sd_angs15_R0 = [fjext.angularity(j, 1.5, jet_R0) for j in all_sd_jets]

CPU times: user 1.86 ms, sys: 116 µs, total: 1.98 ms
Wall time: 2.18 ms


In [18]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False)
ax0, ax1, = axes.flatten()
n, bins, patches = ax0.hist(pts, 25, density=1, facecolor='blue', alpha=0.3, label='anti-$k_{T}$ R=0.4')
n, bins, patches = ax0.hist(sd_pts, 25, density=1, facecolor='red', alpha=0.3, label='Soft Dropped (SD)')
# n, bins, patches = ax0.hist(sd_pts, 25, density=1, facecolor='red', alpha=0.3)
ax0.set_xlabel(r'$p_{T}$ (GeV)')
ax0.set_ylabel('Probability within $\hat{p_{T}} > 20$')
ax0.set_title(r'$\mathrm{PYTHIA\ jets}\ \sqrt{s}=8\ \mathrm{TeV}$ proton-proton')
ax0.grid(True)
ax0.legend(prop={'size': 10})
ax0.set_yscale('log')

n, bins, patches = ax1.hist(sd_delta_pt, 25, density=1, facecolor='green', alpha=0.3, label='$\Delta p_{T} = p_{T}^{SD} - p_{T}$')
ax1.legend(prop={'size': 10})
ax1.grid(True)
ax1.set_yscale('log')
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
fig1, axes1 = plt.subplots(nrows=3, ncols=2, sharex=False, sharey=False)
ax10, ax11, ax12, ax13, ax14, ax15= axes1.flatten()

ax10.set_title(r'angularity $\alpha = 0$')
n, bins, patches = ax10.hist(angs0, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax10.hist(sd_angs0, 25, density=1, facecolor='red', alpha=0.3)

ax11.set_title(r'scaled by $R_{0}$')
n, bins, patches = ax11.hist(angs0_R0, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax11.hist(sd_angs0_R0, 25, density=1, facecolor='red', alpha=0.3)

ax12.set_title(r'angularity $\alpha = 1$')
n, bins, patches = ax12.hist(angs1, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax12.hist(sd_angs1, 25, density=1, facecolor='red', alpha=0.3)

ax13.set_title(r'scaled by $R_{0}$')
n, bins, patches = ax13.hist(angs1_R0, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax13.hist(sd_angs1_R0, 25, density=1, facecolor='red', alpha=0.3)

ax14.set_title(r'angularity $\alpha = 1.5$')
n, bins, patches = ax14.hist(angs15, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax14.hist(sd_angs15, 25, density=1, facecolor='red', alpha=0.3)

ax15.set_title(r'scaled by $R_{0}$')
n, bins, patches = ax15.hist(angs15_R0, 25, density=1, facecolor='blue', alpha=0.3)
n, bins, patches = ax15.hist(sd_angs15_R0, 25, density=1, facecolor='red', alpha=0.3)

fig1.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …